In [21]:
# -----------------------------------------------------------------------------
# 1.  Tabla física de conversión  (no hay cálculos aquí, sólo datos de referencia)
# -----------------------------------------------------------------------------
CONVERSION = {
    # clave : {nombre, unidad_base, pci (MJ/unidad_SI)}
    # -------------------------------------------------------------------------
    "crudo": dict(
        nombre="Petróleo crudo (PCI ≈ 41.9 MJ/kg)",
        unidad_base="kt",
        PCI_MJ_per_kg=41.9,     # 41.9 MJ por kilogramo
    ),
    "gas_nat": dict(
        nombre="Gas natural (PCI ≈ 34 MJ/m³)",
        unidad_base="MMm3",
        PCI_MJ_per_m3=34.0,     # 34 MJ por metro cúbico
    ),
    "bagazo": dict(
        nombre="Bagazo de caña (PCI ≈ 17 MJ/kg seco 50 % humedad)",
        unidad_base="kt",
        PCI_MJ_per_kg=17.0,
    ),
    "lena": dict(
        nombre="Leña (densidad 0.29 t/m³, PCI 15 MJ/kg)",
        unidad_base="Mm3",
        densidad_t_per_m3=0.29,
        PCI_MJ_per_kg=15.0,
    ),
    "carbon_veg": dict(
        nombre="Carbón vegetal (PCI ≈ 30 MJ/kg)",
        unidad_base="kt",
        PCI_MJ_per_kg=30.0,
    ),
    # ------------- Derivados importados --------------------------------------
    "glp": dict(
        nombre="GLP (PCI ≈ 49.7 MJ/kg)",
        unidad_base="kt",
        PCI_MJ_per_kg=49.7,
    ),
    "gasolina": dict(
        nombre="Gasolina (PCI ≈ 41.4 MJ/kg)",
        unidad_base="kt",
        PCI_MJ_per_kg=41.4,
    ),
    "diesel": dict(
        nombre="Diésel (PCI ≈ 42.8 MJ/kg)",
        unidad_base="kt",
        PCI_MJ_per_kg=42.8,
    ),
    "fueloil": dict(
        nombre="Fuel oil (PCI ≈ 41.9 MJ/kg)",
        unidad_base="kt",
        PCI_MJ_per_kg=41.9,
    ),
    # ------------- Electricidad (no requiere conversión) ---------------------
    "electricidad": dict(
        nombre="Electricidad",
        unidad_base="GWh",
        PCI_MJ_per_GWh=3_600_000,   # 1 GWh = 3.6×10⁶ MJ
    ),
}


In [22]:
# datos_cuba_energia.py
# ---------------------
# Factores de conversión (PCI aprox.) y valores del Anuario
# 2019-2022 tal como aparecen en las tablas 10.xx y 13.xx
# Unidades: kt, MMm³, Mm³, GWh

# -----------------------------------------------------------------------------
# 2.  Función que convierte la entrada física → factor_GWh
# -----------------------------------------------------------------------------
def factor_GWh(clave: str) -> float:
    """Devuelve cuántos GWh hay por unidad_base del portador."""
    d = CONVERSION[clave]
    u = d["unidad_base"]

    if u == "kt":        # 1 kt = 10^6 kg
        PCI_MJ = d["PCI_MJ_per_kg"] * 1_000_000
    elif u == "MMm3":    # 1 MMm3 = 10^6 m^3
        PCI_MJ = d["PCI_MJ_per_m3"] * 1_000_000
    elif u == "Mm3":     # leña: densidad * PCI * 10^6 m^3
        masa_t = d["densidad_t_per_m3"] * 1_000_000  # toneladas en 1 Mm³
        PCI_MJ = masa_t * d["PCI_MJ_per_kg"]
    elif u == "GWh":
        return 1.0
    else:
        raise ValueError(f"Unidad desconocida: {u}")

    # MJ → TWh:  1 GWh = 3.6×10⁶ MJ  ⇒  MJ / 3.6×10⁶ = GWh
    return PCI_MJ / 3_600_000


# -----------------------------------------------------------------------------
# 3.  Genera el diccionario FACT automáticamente
# -----------------------------------------------------------------------------
FACT = {k: round(factor_GWh(k), 3) for k in CONVERSION}


# FACT = {
#     "crudo"     : 11.63,   # GWh/kt  (≈ 41.9 GJ/t)
#     "gas_nat"   :  9.50,   # GWh/MMm³
#     "bagazo"    :  4.75,   # GWh/kt
#     "lena"      :1200.0,   # GWh/Mm³
#     "carbon_veg":  8.30,   # GWh/kt
#     "glp"       : 13.80,   # GWh/kt
#     "gasolina"  : 11.50,   # GWh/kt
#     "diesel"    : 11.90,   # GWh/kt
#     "fueloil"   : 11.63,   # GWh/kt
# }

FACT

{'crudo': 11.639,
 'gas_nat': 9.444,
 'bagazo': 4.722,
 'lena': 1.208,
 'carbon_veg': 8.333,
 'glp': 13.806,
 'gasolina': 11.5,
 'diesel': 11.889,
 'fueloil': 11.639,
 'electricidad': 1.0}

In [23]:
PORTADORES = {
    # ───────────────────────────────────────────────
    # PRODUCCIÓN NACIONAL  (fuente: “producción”)
    # ───────────────────────────────────────────────
    "crudo": dict(
        nombre="Petróleo crudo",
        familia="petroleo",
        unidad="kt",
        factor_GWh=FACT["crudo"],               # 11.63 GWh/kt
        datos={
            2019: dict(producción=2371.5, importación=2415.9),   # tabla 10.2 / 10.6
            2020: dict(producción=2320.2, importación=1793.4),
            2021: dict(producción=2433.3, importación=2305.3),
            2022: dict(producción=2373.9, importación=2394.7),
        },
    ),

    "gas_nat": dict(
        nombre="Gas natural",
        familia="gas",
        unidad="MMm³",
        factor_GWh=FACT["gas_nat"],             # 9.50 GWh/MMm³
        datos={
            2019: dict(producción=951.1, importación=0.0),
            2020: dict(producción=894.8, importación=0.0),
            2021: dict(producción=651.9, importación=0.0),
            2022: dict(producción=843.5, importación=0.0),
        },
    ),

    "bagazo": dict(
        nombre="Bagazo de caña",
        familia="biomasa",
        unidad="kt",
        factor_GWh=FACT["bagazo"],              # 4.75 GWh/kt
        datos={
            2019: dict(producción=5195.3, importación=0.0),
            2020: dict(producción=4045.3, importación=0.0),
            2021: dict(producción=3607.4, importación=0.0),
            2022: dict(producción=2551.2, importación=0.0),
        },
    ),

    "lena": dict(
        nombre="Leña",
        familia="biomasa",
        unidad="Mm³",
        factor_GWh=FACT["lena"],                # 1200 GWh/Mm³
        datos={
            2019: dict(producción=17.2, importación=0.0),
            2020: dict(producción=16.7, importación=0.0),
            2021: dict(producción=16.2, importación=0.0),
            2022: dict(producción=14.0, importación=0.0),
        },
    ),

    "carbon_veg": dict(
        nombre="Carbón vegetal",
        familia="biomasa",
        unidad="kt",
        factor_GWh=FACT["carbon_veg"],          # 8.30 GWh/kt
        datos={
            2019: dict(producción=70.4, importación=0.0),
            2020: dict(producción=70.4, importación=0.0),
            2021: dict(producción=56.3, importación=0.0),
            2022: dict(producción=37.5, importación=0.0),
        },
    ),

    "elec_ren": dict(
        nombre="Renovables eléctricas (FV+Eólica+Hidro)",
        familia="electricidad",
        unidad="GWh",
        factor_GWh=1.0,
        datos={
            2019: dict(producción=115.7 + 250.5 + 124.5, importación=0.0),  # 490.7
            2020: dict(producción=111.9 + 260.8 + 119.7, importación=0.0),  # 492.4
            2021: dict(producción=119.7 + 265.0 + 119.7, importación=0.0),  # 504.4
            2022: dict(producción=120.2 + 241.5 + 115.7, importación=0.0),  # 477.4
        },
    ),

    # ───────────────────────────────────────────────
    # IMPORTACIONES DE DERIVADOS
    # ───────────────────────────────────────────────
    "glp": dict(
        nombre="GLP",
        familia="gas",
        unidad="kt",
        factor_GWh=FACT["glp"],
        datos={
            2019: dict(producción=0.0, importación=119.2),
            2020: dict(producción=0.0, importación=111.7),
            2021: dict(producción=0.0, importación=152.1),
            2022: dict(producción=0.0, importación=187.5),
        },
    ),

    "gasolina": dict(
        nombre="Gasolina",
        familia="petroleo",
        unidad="kt",
        factor_GWh=FACT["gasolina"],
        datos={
            2019: dict(producción=0.0, importación=236.3),
            2020: dict(producción=0.0, importación=226.4),
            2021: dict(producción=0.0, importación=125.8),
            2022: dict(producción=0.0, importación=380.7),
        },
    ),

    "diesel": dict(
        nombre="Diésel",
        familia="petroleo",
        unidad="kt",
        factor_GWh=FACT["diesel"],
        datos={
            2019: dict(producción=0.0, importación=729.8),
            2020: dict(producción=0.0, importación=539.1),
            2021: dict(producción=0.0, importación=956.7),
            2022: dict(producción=0.0, importación=876.0),
        },
    ),

    "fueloil": dict(
        nombre="Fuel oil",
        familia="fueloil",
        unidad="kt",
        factor_GWh=FACT["fueloil"],
        datos={
            2019: dict(producción=0.0, importación=1342.8),
            2020: dict(producción=0.0, importación=1935.0),
            2021: dict(producción=0.0, importación=1124.6),
            2022: dict(producción=0.0, importación=679.2),
        },
    ),

    "elec_imp": dict(
        nombre="Electricidad importada",
        familia="electricidad",
        unidad="GWh",
        factor_GWh=1.0,
        datos={
            2019: dict(producción=0.0, importación=0.0),
            2020: dict(producción=0.0, importación=0.0),
            2021: dict(producción=0.0, importación=0.0),
            2022: dict(producción=0.0, importación=2590.7),
        },
    ),
}





GWH_PER_PJ = 1_000 / 3.6           # ≃ 277.777…

def energia(port, año, fuente="producción", unidad="GWh"):
    """Devuelve la energía de un portador en GWh o PJ."""
    gwh = PORTADORES[port]["datos"][año][fuente] * PORTADORES[port]["factor_GWh"]
    if unidad.lower() == "pj":
        return gwh / GWH_PER_PJ
    return gwh      # por defecto GWh


def energia_total(port, año, unidad="GWh"):
    vals    = PORTADORES[port]["datos"][año]
    gwh_tot = (vals["producción"] + vals["importación"]) * PORTADORES[port]["factor_GWh"]
    return gwh_tot / GWH_PER_PJ if unidad.lower() == "pj" else gwh_tot

In [24]:
PORTADORES

{'crudo': {'nombre': 'Petróleo crudo',
  'familia': 'petroleo',
  'unidad': 'kt',
  'factor_GWh': 11.639,
  'datos': {2019: {'producción': 2371.5, 'importación': 2415.9},
   2020: {'producción': 2320.2, 'importación': 1793.4},
   2021: {'producción': 2433.3, 'importación': 2305.3},
   2022: {'producción': 2373.9, 'importación': 2394.7}}},
 'gas_nat': {'nombre': 'Gas natural',
  'familia': 'gas',
  'unidad': 'MMm³',
  'factor_GWh': 9.444,
  'datos': {2019: {'producción': 951.1, 'importación': 0.0},
   2020: {'producción': 894.8, 'importación': 0.0},
   2021: {'producción': 651.9, 'importación': 0.0},
   2022: {'producción': 843.5, 'importación': 0.0}}},
 'bagazo': {'nombre': 'Bagazo de caña',
  'familia': 'biomasa',
  'unidad': 'kt',
  'factor_GWh': 4.722,
  'datos': {2019: {'producción': 5195.3, 'importación': 0.0},
   2020: {'producción': 4045.3, 'importación': 0.0},
   2021: {'producción': 3607.4, 'importación': 0.0},
   2022: {'producción': 2551.2, 'importación': 0.0}}},
 'lena': {'

In [6]:
# --------------------------------------------------------------------
# Función única y flexible
# --------------------------------------------------------------------
GWH_PER_PJ = 1_000 / 3.6            # 277.777… GWh en 1 PJ
GWH_PER_TWH = 1_000                 # 1 TWh = 1 000 GWh
def energia(portador_key: str, año: int,
            fuente: str = "total",   # "producción" | "importación" | "total"
            unidad: str = "GWh"):    # "GWh" | "PJ"
    """
    Devuelve la energía del portador en la unidad deseada.

    Parámetros
    ----------
    portador_key : str
        Clave del portador en PORTADORES, p.e. "crudo", "gas_nat".
    año : int
        Año (2019‑2022 en nuestro diccionario)
    fuente : {"producción", "importación", "total"}
        - "producción"   → solo producción nacional
        - "importación"  → solo importaciones
        - "total"        → producción + importaciones
    unidad : {"GWh", "PJ"}
        Unidad de salida.

    Retorna
    -------
    float
        Valor de la energía en la unidad solicitada.
    """
    p = PORTADORES[portador_key]
    datos = p["datos"][año]

    if fuente not in ("producción", "importación", "total"):
        raise ValueError("fuente debe ser 'producción', 'importación' o 'total'")

    # Cantidad física (kt, MMm³, Mm³, GWh…)
    if fuente == "total":
        cantidad = datos["producción"] + datos["importación"]
    else:
        cantidad = datos[fuente]

    # Conversión a GWh
    energia_GWh = cantidad * p["factor_GWh"]
    # Conversión a TWh


    # Conversión final, si se pidió PJ
    if unidad.lower() == "pj":
        return energia_GWh / GWH_PER_PJ
    elif unidad.lower() == "twh":
        return energia_GWh / GWH_PER_TWH
    return energia_GWh


### Llamar los datos del diccionario
```python
PORTADORES["crudo"]["datos"][2022]["producción"]+PORTADORES["crudo"]["datos"][2022]["importación"]

In [7]:
PORTADORES["crudo"]["datos"][2022]["producción"]+PORTADORES["crudo"]["datos"][2022]["importación"]

4768.6

In [8]:
39.1+10.6+1157.3+4240.4+10.7+179.1+3+343.9+47.7+59.9+151.1


6242.799999999999

### Ejecutar de la siguiente forma
```python
print("Crudo total 2022:", energia("crudo", 2022, "total", "PJ"), "PJ")
print("Crudo total 2022:", energia("crudo", 2022, "total", "GWh"), "GWh")
print("Crudo total 2019:", energia("crudo", 2022, "importación", "PJ"), "PJ")

print("Crudo total 2019:", energia("crudo", 2022, "producción", "PJ"), "PJ")

In [9]:
print("Crudo total 2022:", energia("crudo", 2022, "total", "PJ"), "PJ")
print("Crudo total 2022:", energia("crudo", 2022, "total", "GWh"), "GWh")
print("Crudo total 2019:", energia("crudo", 2019, "total", "TWh"), "TWh")

print("Crudo producción 2019:", energia("crudo", 2019, "producción", "TWh"), "TWh")
print("Crudo producción 2022:", energia("crudo", 2022, "producción", "PJ"), "PJ")
print("Crudo importación 2022:", energia("crudo", 2022, "importación", "PJ"), "PJ")

Crudo total 2022: 199.80624744 PJ
Crudo total 2022: 55501.7354 GWh
Crudo total 2019: 55.720548599999994 TWh
Crudo producción 2019: 27.601888499999998 TWh
Crudo producción 2022: 99.46735956 PJ
Crudo importación 2022: 100.33888787999999 PJ


In [10]:
# sankey_utils.py
import plotly.graph_objects as go
from itertools import chain

# ---------- Colores ----------
COLOR_FAM = {
    "petroleo": "#6e6e6e",
    "gas": "#ff8c00",
    "biomasa": "#4caf50",
    "electricidad": "#fffb00",
    "fueloil": "#d7301f",
    "perdidas": "#c7c7c7",
}
COLOR_NODE = {
    "Red eléctrica bruta": "#1f77b4",
    "Electricidad disponible": "#64b5f6",
    "Pérdidas eléctricas": "#c7c7c7",
}

def rgba(hexcol, a=0.7):
    hexcol = hexcol.lstrip("#")
    r,g,b = int(hexcol[:2],16), int(hexcol[2:4],16), int(hexcol[4:],16)
    return f"rgba({r},{g},{b},{a})"

# ---------- Construcción de flujos simplificada ----------
def build_flows(year_data, FACT):
    d = year_data
    flows = []

    # producción nacional
    flows += [
        ("Producción nacional","Crudo nacional", d["crudo_kt"]*FACT["crudo"]),
        ("Producción nacional","Gas natural",    d["gas_MM3"]*FACT["gas_nat"]),
        ("Producción nacional","Bagazo",         d["bagazo_kt"]*FACT["bagazo"]),
        ("Producción nacional","Leña",           d["lena_Mm3"]*FACT["lena"]),
        ("Producción nacional","Carbón vegetal", d["carbon_kt"]*FACT["carbon_veg"]),
        ("Producción nacional","Renovables eléctricas", d["elec_ren_GWh"]),
    ]

    # importaciones
    flows += [
        ("Importaciones","GLP importado",     d["glp_imp_kt"]*FACT["glp"]),
        ("Importaciones","Gasolina importada",d["gasol_imp_kt"]*FACT["gasolina"]),
        ("Importaciones","Diésel importado",  d["diesel_imp_kt"]*FACT["diesel"]),
        ("Importaciones","Fuel oil importado",d["fo_imp_kt"]*FACT["fueloil"]),
        ("Importaciones","Electricidad importada", d["elec_imp_GWh"]),
    ]

    # rules 60/30/10 pool líquidos
    liq_pool = (
        d["gasol_imp_kt"]*FACT["gasolina"] +
        d["diesel_imp_kt"]*FACT["diesel"] +
        d["fo_imp_kt"]*FACT["fueloil"] +
        d["crudo_kt"]*FACT["crudo"]*0.60
    )
    flows += [
        ("Crudo nacional","Pool líquidos", d["crudo_kt"]*FACT["crudo"]*0.60),
        ("Gasolina importada","Pool líquidos", d["gasol_imp_kt"]*FACT["gasolina"]),
        ("Diésel importado","Pool líquidos",   d["diesel_imp_kt"]*FACT["diesel"]),
        ("Fuel oil importado","Pool líquidos", d["fo_imp_kt"]*FACT["fueloil"]),
        ("Pool líquidos","Transporte",         liq_pool*0.60),
        ("Pool líquidos","Generación térmica", liq_pool*0.30),
        ("Pool líquidos","Industria",          liq_pool*0.10),
    ]

    # gas 80/20
    flows += [
        ("Gas natural","Generación térmica", d["gas_MM3"]*FACT["gas_nat"]*0.80),
        ("Gas natural","Red gas",            d["gas_MM3"]*FACT["gas_nat"]*0.20),
        ("GLP importado","Red gas",          d["glp_imp_kt"]*FACT["glp"]),
        ("Red gas","Residencial",            (d["glp_imp_kt"]*FACT["glp"] + d["gas_MM3"]*FACT["gas_nat"]*0.20)*0.90),
        ("Red gas","Servicios",              (d["glp_imp_kt"]*FACT["glp"] + d["gas_MM3"]*FACT["gas_nat"]*0.20)*0.10),
    ]

    # biomasa trad
    flows += [
        ("Leña","Residencial", d["lena_Mm3"]*FACT["lena"]),
        ("Carbón vegetal","Residencial", d["carbon_kt"]*FACT["carbon_veg"]),
    ]

    # bagazo 70/30
    flows += [
        ("Bagazo","Industria", d["bagazo_kt"]*FACT["bagazo"]*0.70),
        ("Bagazo","Bioeléctrica", d["bagazo_kt"]*FACT["bagazo"]*0.30),
        ("Bioeléctrica","Red eléctrica bruta", d["bagazo_kt"]*FACT["bagazo"]*0.30*0.20),
    ]

    # renovables + importada
    flows += [
        ("Renovables eléctricas","Red eléctrica bruta", d["elec_ren_GWh"]),
        ("Electricidad importada","Red eléctrica bruta", d["elec_imp_GWh"]),
    ]

    # gen térmica neta
    gen_term = d["elec_total_GWh"] - d["elec_ren_GWh"] - d["elec_imp_GWh"]
    flows += [("Generación térmica","Red eléctrica bruta", gen_term)]

    # red → pérdidas + dispo
    flows += [
        ("Red eléctrica bruta","Pérdidas eléctricas", d["elec_losses_GWh"]),
        ("Red eléctrica bruta","Electricidad disponible",
         d["elec_total_GWh"] - d["elec_losses_GWh"]),
    ]

    # disponible → reparto simplificado
    transele = 232 if d["elec_total_GWh"] < 19000 else 200
    flows += [
        ("Electricidad disponible","Residencial", d["elec_res_GWh"]),
        ("Electricidad disponible","Servicios", 1200),
        ("Electricidad disponible","Industria", 1000),
        ("Electricidad disponible","Transporte", transele),
    ]
    return flows

# ---------- Función para construir Sankey con herencia de color ----------
def build_sankey(flows, title):
    nodes = list({n for n in chain.from_iterable((f[:2] for f in flows))})
    idx = {n:i for i,n in enumerate(nodes)}

    # nodo → color
    node_col = {}
    def fam(name):
        name_l = name.lower()
        if "pérdidas" in name: return "perdidas"
        if "eléctric" in name_l: return "electricidad"
        if "gas" in name_l: return "gas"
        if any(x in name_l for x in ("bagazo","leña","carbón")): return "biomasa"
        if any(x in name_l for x in ("fuel","gasolina","diésel","crudo","pool")): return "petroleo"
        return "electricidad"

    for n in nodes:
        node_col[n] = COLOR_NODE.get(n, COLOR_FAM.get(fam(n),"#bbbbbb"))

    src=tgt=val=lcol=[]; src=[]; tgt=[]; val=[]; lcol=[]
    for s,t,v in flows:
        if v<=0: continue
        src.append(idx[s]); tgt.append(idx[t]); val.append(round(v/1000,3))
        lcol.append(rgba(node_col[s],0.65))

    fig = go.Figure(data=[go.Sankey(
        arrangement="snap",
        node=dict(label=nodes, pad=18, thickness=20,
                  color=[node_col[n] for n in nodes]),
        link=dict(source=src, target=tgt, value=val, color=lcol,
                  hovertemplate='%{source.label} → %{target.label}<br>%{value:.3f} TWh<extra></extra>'),
    )])
    fig.update_layout(title_text=title, font_size=11,
                      width=1400, height=800)
    return fig


In [11]:
# run_sankey.py
from pathlib import Path
import plotly.io as pio


pio.renderers.default = "browser"   # abre en navegador; cambia a 'notebook_connected' si prefieres inline

try:
    out_dir = Path(__file__).parent          # funciona cuando es script
except NameError:
    out_dir = Path.cwd()    

for year, data_dict in DATA.items():
    flows = build_flows(data_dict, FACT)
    fig = build_sankey(flows, f"Balance energético de Cuba – {year}")
    html_file = out_dir / f"sankey_cuba_{year}.html"
    fig.write_html(html_file, include_plotlyjs="cdn", full_html=True)
    print(f"Gráfico {year} listo → {html_file}")

print("✔  Todos los Sankey generados.")


NameError: name 'DATA' is not defined

In [15]:
import pandas as pd
# from ace_tools import display_dataframe_to_user

# -------- Tabla 10.1 completa (Minería y Energía) ----------
tabla_10_1 = pd.DataFrame({
    "Producto / División": [
        # --- Extracción de petróleo crudo y gas natural ---
        "Gas natural",
        "Petróleo crudo",
        # --- Extracción y beneficio de mineral de níquel ---
        "Laterita + serpentina niquelífera",
        # --- Explotación de otras minas y canteras ---
        "Arcilla para cemento",
        "Arcilla para cerámica roja",
        "Arena aluvial",
        "Arena sílice",
        "Bentonita",
        "Caliza (industria cemento)",
        "Caolín",
        "Cieno carbonatado",
        "Feldespato",
        "Fosforita",
        "Margas (industria cemento)",
        "Mármol",
        "Piedra de cantería",
        "Piedra para relleno",
        "Piedra para trituración",
        "Sal en grano (extracción)",
        "Tobas para cemento",
        "Yeso",
        "Zeolita",
    ],
    "Unidad": [
        "MMm³","Mt",
        "Mt",
        "MMt","Mm³","Mm³","Mt","t",
        "MMt","Mt","Mt","Mt","Mt","MMt","Mm³","Mm³","Mm³","Mm³","Mt","Mt","Mt","Mt"
    ],
    2019: [
        951.1, 2371.5, 4511.1,
        0.2,22.4,62.7,19.7,504.3,0.8,2.1,651.1,2.1,None,0.0,9.5,0.0,20.7,1652.2,210.4,38.8,66.3,120.8
    ],
    2020: [
        894.8, 2320.2, 5046.1,
        0.2,15.9,25.5,22.5,486.0,0.5,2.0,631.4,14.1,1.6,0.0,11.0,0.1,52.6,1661.8,226.0,38.8,62.3,103.3
    ],
    2021: [
        863.5, 2433.3, 5145.0,
        0.1,11.8,14.9,14.6,517.0,0.4,1.4,655.0,17.8,1.4,None,12.7,0.0,14.9,1165.1,204.6,17.9,53.2,83.4
    ],
    2022: [
        843.5, 2373.9, 2546.8,
        0.1,5.4,8.9,6.9,289.5,0.0,0.5,743.2,5.9,1.1,None,4.1,0.1,9.9,1041.1,195.6,17.3,32.6,77.5
    ],
    2023: [
        955.0, None, None,
        0.1,7.3,4.8,10.1,269.4,0.0,1.5,632.2,17.3,0.0,None,12.6,0.0,2.0,763.7,87.9,20.7,37.4,67.8
    ],
}).set_index("Producto / División")
tabla_10_1.T


Producto / División,Gas natural,Petróleo crudo,Laterita + serpentina niquelífera,Arcilla para cemento,Arcilla para cerámica roja,Arena aluvial,Arena sílice,Bentonita,Caliza (industria cemento),Caolín,...,Fosforita,Margas (industria cemento),Mármol,Piedra de cantería,Piedra para relleno,Piedra para trituración,Sal en grano (extracción),Tobas para cemento,Yeso,Zeolita
Unidad,MMm³,Mt,Mt,MMt,Mm³,Mm³,Mt,t,MMt,Mt,...,Mt,MMt,Mm³,Mm³,Mm³,Mm³,Mt,Mt,Mt,Mt
2019,951.1,2371.5,4511.1,0.2,22.4,62.7,19.7,504.3,0.8,2.1,...,NaN,0.0,9.5,0.0,20.7,1652.2,210.4,38.8,66.3,120.8
2020,894.8,2320.2,5046.1,0.2,15.9,25.5,22.5,486.0,0.5,2.0,...,1.6,0.0,11.0,0.1,52.6,1661.8,226.0,38.8,62.3,103.3
2021,863.5,2433.3,5145.0,0.1,11.8,14.9,14.6,517.0,0.4,1.4,...,1.4,NaN,12.7,0.0,14.9,1165.1,204.6,17.9,53.2,83.4
2022,843.5,2373.9,2546.8,0.1,5.4,8.9,6.9,289.5,0.0,0.5,...,1.1,NaN,4.1,0.1,9.9,1041.1,195.6,17.3,32.6,77.5
2023,955.0,NaN,NaN,0.1,7.3,4.8,10.1,269.4,0.0,1.5,...,0.0,NaN,12.6,0.0,2.0,763.7,87.9,20.7,37.4,67.8


In [28]:
# -------- Tabla 10.2 · Producción nacional de energía ----------
tabla_10_2_energia = pd.DataFrame({
    "Año": [2019, 2020, 2021, 2022, 2023],
    "Cantidad_Primaria_Mtcc":   [4765.2, 4368.0, 4354.7, 3990.1, 3773.8],
    "Cantidad_Secundaria_Mtcc": [4226.7, 3286.2, 1268.4, 1104.3, 1066.6],
    "Estructura_Primaria_%":    [53.0, 57.1, 77.4, 78.3, 78.0],
    "Estructura_Secundaria_%":  [47.0, 42.9, 22.6, 21.7, 22.0],
    "Tasa_Primaria_%":          [-5.4, -8.3, -0.3, -8.4, -5.4],
    "Tasa_Secundaria_%":        [ 1.9, -22.3, -61.4, -12.9, -3.4],
}).set_index("Año")
tabla_10_2_energia

,Cantidad_Primaria_Mtcc,Cantidad_Secundaria_Mtcc,Estructura_Primaria_%,Estructura_Secundaria_%,Tasa_Primaria_%,Tasa_Secundaria_%
Año,,,,,,
2019,4765.2,4226.7,53.0,47.0,-5.4,1.9
2020,4368.0,3286.2,57.1,42.9,-8.3,-22.3
2021,4354.7,1268.4,77.4,22.6,-0.3,-61.4
2022,3990.1,1104.3,78.3,21.7,-8.4,-12.9
2023,3773.8,1066.6,78.0,22.0,-5.4,-3.4


In [ ]:
# -------- Tabla 10.3 · Producción nacional de energía primaria ----------
tabla_10_3 = pd.DataFrame({
    "Año":   [2019,    2020,    2021,    2022,    2023],
    "Petróleo (Mt)":          [2371.5, 2320.2, 2433.3, 2373.9, None],
    "Gas natural (MMm³)":     [ 951.1,  894.8,  651.9,  843.5,  955.0],
    "Hidro‑energía (GWh)":    [ 124.5,  111.9,  119.7,  120.2,  106.5],
    "Leña (Mm³)":             [ 923.9,  813.6,  797.7,  659.1,  590.8],
    "Productos de caña (Mt)": [5486.4, 5863.5, 4407.5, 3217.9, 2222.7],
    "Bagazo (Mt)":            [5195.3, 4045.3, 3607.4, 2551.2, 1556.0],
}).set_index("Año")

tabla_10_3


,Petróleo (Mt),Gas natural (MMm³),Hidro‑energía (GWh),Leña (Mm³),Productos de caña (Mt),Bagazo (Mt)
Año,,,,,,
2019,2371.5,951.1,124.5,923.9,5486.4,5195.3
2020,2320.2,894.8,111.9,813.6,5863.5,4045.3
2021,2433.3,651.9,119.7,797.7,4407.5,3607.4
2022,2373.9,843.5,120.2,659.1,3217.9,2551.2
2023,NaN,955.0,106.5,590.8,2222.7,1556.0


In [31]:
# ---------------- Tabla 10.4 · Producción nacional de energía secundaria ----------------
tabla_10_4 = pd.DataFrame({
    "Electricidad (GWh)":          [20705.6, 19070.9, 17965.5, 15732.1, 15331.1],
    "Derivados del petróleo (Mt)": [1701.3,  2052.0,  1461.4,  1618.9,   None],
    "Carbón vegetal (Mt)":         [  76.1,    75.6,    59.2,    39.4,    65.5],
    "Alcohol desnaturalizado (Mhl)": [173.8, 171.3, 148.2, 144.6, 117.9],
    "Gas manufacturado (MMm³)":    [185.4, 187.6, 183.9, 181.2, 175.0],
}, index=[2019, 2020, 2021, 2022, 2023])
tabla_10_4

,Electricidad (GWh),Derivados del petróleo (Mt),Carbón vegetal (Mt),Alcohol desnaturalizado (Mhl),Gas manufacturado (MMm³)
2019,20705.6,1701.3,76.1,173.8,185.4
2020,19070.9,2052.0,75.6,171.3,187.6
2021,17965.5,1461.4,59.2,148.2,183.9
2022,15732.1,1618.9,39.4,144.6,181.2
2023,15331.1,NaN,65.5,117.9,175.0


In [36]:
# ------------- Tabla 10.5 · Producción de derivados del petróleo -------------
tabla_10_5 = pd.DataFrame({
    "Año": [2018, 2019, 2020, 2021, 2022],
    "GLP (kt)":                         [29.9,   21.6,   38.9,   27.8,   27.5],
    "Gasolina motor (kt)":              [212.0,  199.2,  291.1,  107.3,  142.1],
    "Queroseno (kt)":                   [6.5,    1.1,    4.2,    3.8,    2.4],
    "Turbocombustible (kt)":            [65.7,   82.8,   37.3,   37.0,   33.7],
    "Combustible diésel (kt)":          [702.7,  447.4,  540.7,  382.4,  338.4],
    "Fuel oil (kt)":                    [1313.3, 766.3,  903.3,  663.8,  928.7],
    "Coque combustible (kt)":           [1.4,    1.0,    4.0,    None,   None],
    "Gas combustible (kt)":             [4.2,    5.3,    14.4,   8.0,    10.7],
    "Nafta industrial + Solventes (kt)":[300.9,  105.1,  148.3,  171.8,  81.3],
    "   Nafta industrial (kt)":         [281.8,  87.3,   133.4,  159.0,  66.1],
    "Aceites/grasas lubricantes (kt)":  [51.1,   33.2,   36.3,   30.7,   25.7],
    "Asfalto de petróleo (kt)":         [63.0,   38.4,   33.5,   28.7,   28.4],
        "Total (kt)":                   [2750.7, 1701.3, 2052.0, 1461.4, 1618.9],
}).set_index("Año")
tabla_10_5

,GLP (kt),Gasolina motor (kt),Queroseno (kt),Turbocombustible (kt),Combustible diésel (kt),Fuel oil (kt),Coque combustible (kt),Gas combustible (kt),Nafta industrial + Solventes (kt),Nafta industrial (kt),Aceites/grasas lubricantes (kt),Asfalto de petróleo (kt),Total (kt)
Año,,,,,,,,,,,,,
2018,29.9,212.0,6.5,65.7,702.7,1313.3,1.4,4.2,300.9,281.8,51.1,63.0,2750.7
2019,21.6,199.2,1.1,82.8,447.4,766.3,1.0,5.3,105.1,87.3,33.2,38.4,1701.3
2020,38.9,291.1,4.2,37.3,540.7,903.3,4.0,14.4,148.3,133.4,36.3,33.5,2052.0
2021,27.8,107.3,3.8,37.0,382.4,663.8,NaN,8.0,171.8,159.0,30.7,28.7,1461.4
2022,27.5,142.1,2.4,33.7,338.4,928.7,NaN,10.7,81.3,66.1,25.7,28.4,1618.9


In [39]:
# -------- Tabla 10.6 · Importaciones de productos energéticos ----------
tabla_10_6 = pd.DataFrame({
    "Año": [2019, 2020, 2021, 2022, 2023],
    "Petróleo crudo (kt)":        [2415.9, 1793.4, 2305.3, 2394.7, None],
    "Carbón mineral (kt)":        [0.0,   2.2,    4.3,    2.3,  None],
    "Derivados del petróleo (kt)":[2810.6, 3084.2, 2562.7, 2342.1, None],
    "Coque de carbón (kt)":       [None,  None,   None,   None, None],
    "Electricidad (GWh)":         [449.4, 1401.6, 1384.7, 2590.7, 4993.8],
    "Tasa Primarios (%)":         [-5.4, -25.7, 28.6, -8.6, None],
    "Tasa Secundarios (%)":       [8.5,  37.6, -12.0, 3.8, None],
}).set_index("Año")
tabla_10_6

,Petróleo crudo (kt),Carbón mineral (kt),Derivados del petróleo (kt),Coque de carbón (kt),Electricidad (GWh),Tasa Primarios (%),Tasa Secundarios (%)
Año,,,,,,,
2019,2415.9,0.0,2810.6,None,449.4,-5.4,8.5
2020,1793.4,2.2,3084.2,None,1401.6,-25.7,37.6
2021,2305.3,4.3,2562.7,None,1384.7,28.6,-12.0
2022,2394.7,2.3,2342.1,None,2590.7,-8.6,3.8
2023,NaN,NaN,NaN,None,4993.8,NaN,NaN


In [40]:
# -------- Tabla 10.7 · Importaciones de derivados del petróleo ----------
tabla_10_7 = pd.DataFrame({
    "Año": [2018, 2019, 2020, 2021, 2022],
    "GLP (kt)":                           [134.6,  119.2,  111.7,  152.1, 187.5],
    "Gasolina motor (kt)":                [254.4,  236.3,  226.4,  125.8, 380.7],
    "Turbocombustible (kt)":              [467.3,  366.8,  234.6,  174.9, 181.7],
    "Combustible diésel (kt)":            [1178.4, 729.8,  539.1,  956.7, 876.0],
    "Fuel oil (kt)":                      [912.0,  1342.8, 1935.0, 1124.6, 679.2],
    "Aceites/grasas lubricantes (kt)":    [3.4,    3.6,    1.3,    4.1,   10.0],
    "Gasolina aviación (kt)":             [6.6,    0.0,    5.2,    2.0,   2.2],
    "Aceites lubricantes bases (kt)":     [45.8,   12.3,   30.9,   22.5,  24.8],
    "Total (kt)":                         [3002.5, 2810.6, 3084.2, 2562.7, 2342.1],
}).set_index("Año")
tabla_10_7

,GLP (kt),Gasolina motor (kt),Turbocombustible (kt),Combustible diésel (kt),Fuel oil (kt),Aceites/grasas lubricantes (kt),Gasolina aviación (kt),Aceites lubricantes bases (kt),Total (kt)
Año,,,,,,,,,
2018,134.6,254.4,467.3,1178.4,912.0,3.4,6.6,45.8,3002.5
2019,119.2,236.3,366.8,729.8,1342.8,3.6,0.0,12.3,2810.6
2020,111.7,226.4,234.6,539.1,1935.0,1.3,5.2,30.9,3084.2
2021,152.1,125.8,174.9,956.7,1124.6,4.1,2.0,22.5,2562.7
2022,187.5,380.7,181.7,876.0,679.2,10.0,2.2,24.8,2342.1


In [41]:
# ------------------ Tabla 10.8 · Consumo de portadores energéticos primarios ------------------
tabla_10_8 = pd.DataFrame({
    "Petróleo crudo (Mt)":        [4691.9, 2610.8, 3002.9, 3032.1, None],
    "Carbón antracita (Mt)":      [1.4,    1.1,    0.2,    0.7,    0.2],
    "Productos de caña (Mt)":     [5427.0, 5763.2, 4355.0, 3175.6, 2204.1],
    "Bagazo (Mt)":                [5154.4, 4048.0, 3600.4, 2545.9, 1574.4],
    "Leña (Mt)":                  [823.8,  811.9,  736.0,  612.0,  548.6],
    "Gas natural (MMm³)":         [917.3,  863.1,  688.5,  892.5,  660.5],
}, index=[2019, 2020, 2021, 2022, 2023])
tabla_10_8.index.name = "Año"


# ------------------ Tabla 10.9 · Consumo de portadores energéticos secundarios ----------------
tabla_10_9 = pd.DataFrame({
    "Derivados del petróleo (Mt)":   [6995.2, 6496.9, 6557.5, 6242.9, None],
    "Carbón vegetal (Mt)":           [71.8,   70.4,   56.3,   37.5,   22.7],
    "Gas manufacturado (MMm³)":      [185.3,  187.3,  180.2,  177.8,  171.6],
    "Alcohol desnaturalizado (Mhl)": [170.2,  162.7,  143.1,  138.8,  132.3],
}, index=[2019, 2020, 2021, 2022, 2023])

In [42]:
tabla_10_8

,Petróleo crudo (Mt),Carbón antracita (Mt),Productos de caña (Mt),Bagazo (Mt),Leña (Mt),Gas natural (MMm³)
Año,,,,,,
2019,4691.9,1.4,5427.0,5154.4,823.8,917.3
2020,2610.8,1.1,5763.2,4048.0,811.9,863.1
2021,3002.9,0.2,4355.0,3600.4,736.0,688.5
2022,3032.1,0.7,3175.6,2545.9,612.0,892.5
2023,NaN,0.2,2204.1,1574.4,548.6,660.5


In [43]:
# ---------------- Tabla 10.10 · Consumo de petróleo crudo y derivados ----------------
tabla_10_10 = pd.DataFrame({
    "Año": [2018, 2019, 2020, 2021, 2022],
    "Total (kt)":                       [7303.4, 6995.2, 6597.9, 6557.5, 6242.9],
    "Aceites/grasas lubricantes (kt)":  [54.1,   49.7,   40.2,   37.3,   39.1],
    "Asfalto (kt)":                     [65.3,   42.6,   36.4,   28.0,   10.6],
    "Combustible diésel (kt)":          [1787.0, 1458.1, 1112.6, 1185.7, 1157.3],
    "Coque combustible (kt)":           [1.4,    1.0,    4.0,    None,   None],
    "Fuel oil (kt)":                    [4769.7, 4833.7, 4793.3, 4686.4, 4240.6],
    "Gas combustible (kt)":             [4.2,    5.3,    14.4,   8.0,    10.7],
    "GLP (kt)":                         [159.8,  170.2,  153.5,  162.3,  179.1],
    "Gasolina aviación (kt)":           [4.3,    2.6,    2.5,    2.5,    3.0],
    "Gasolina motor (kt)":              [254.3,  232.1,  203.9,  250.2,  343.9],
    "Nafta industrial + Solventes (kt)":[48.6,   44.5,   40.3,   48.3,   47.7],
    "Queroseno (kt)":                   [80.0,   73.4,   74.0,   62.1,   59.9],
    "Turbo combustible (kt)":           [74.8,   82.1,   122.4,  86.7,   151.1],
}).set_index("Año")

tabla_10_10

,Total (kt),Aceites/grasas lubricantes (kt),Asfalto (kt),Combustible diésel (kt),Coque combustible (kt),Fuel oil (kt),Gas combustible (kt),GLP (kt),Gasolina aviación (kt),Gasolina motor (kt),Nafta industrial + Solventes (kt),Queroseno (kt),Turbo combustible (kt)
Año,,,,,,,,,,,,,
2018,7303.4,54.1,65.3,1787.0,1.4,4769.7,4.2,159.8,4.3,254.3,48.6,80.0,74.8
2019,6995.2,49.7,42.6,1458.1,1.0,4833.7,5.3,170.2,2.6,232.1,44.5,73.4,82.1
2020,6597.9,40.2,36.4,1112.6,4.0,4793.3,14.4,153.5,2.5,203.9,40.3,74.0,122.4
2021,6557.5,37.3,28.0,1185.7,NaN,4686.4,8.0,162.3,2.5,250.2,48.3,62.1,86.7
2022,6242.9,39.1,10.6,1157.3,NaN,4240.6,10.7,179.1,3.0,343.9,47.7,59.9,151.1


In [45]:
import pandas as pd

# Helper to build multiindex columns
cols = [
    ("Aceites/grasas lubricantes", "Total"),
    ("Aceites/grasas lubricantes", "Agricultura"),
    ("Aceites/grasas lubricantes", "Minas y canteras"),
    ("Aceites/grasas lubricantes", "Industria azucarera"),
    ("Aceites/grasas lubricantes", "Ind. manufactureras"),
    ("Aceites/grasas lubricantes", "Construcción"),
    ("Aceites/grasas lubricantes", "Comercio"),
    ("Aceites/grasas lubricantes", "Transporte"),
    ("Aceites/grasas lubricantes", "Servicios empresariales"),
    ("Aceites/grasas lubricantes", "Población"),
    
    ("Asfalto", "Total"),
    ("Asfalto", "Ind. manufactureras"),
    ("Asfalto", "Construcción"),
    
    ("Diésel", "Total"),
    ("Diésel", "Agricultura"),
    ("Diésel", "Minas y canteras"),
    ("Diésel", "Industria azucarera"),
    ("Diésel", "Ind. manufactureras"),
    ("Diésel", "Suministro elec/gas/agua"),
    ("Diésel", "Construcción"),
    ("Diésel", "Comercio"),
    ("Diésel", "Transporte"),
    ("Diésel", "Servicios empresariales"),
    ("Diésel", "Administración pública"),
    
    ("Coque combustible", "Total"),
    ("Coque combustible", "Ind. manufactureras"),
    
    ("Fuel oil", "Total"),
    ("Fuel oil", "Minas y canteras"),
    ("Fuel oil", "Industria azucarera"),
    ("Fuel oil", "Ind. manufactureras"),
    ("Fuel oil", "Suministro elec/gas/agua"),
    ("Fuel oil", "Construcción"),
    
    ("Gas combustible", "Total"),
    ("Gas combustible", "Ind. manufactureras"),
    
    ("GLP", "Total"),
    ("GLP", "Minas y canteras"),
    ("GLP", "Ind. manufactureras"),
    ("GLP", "Comercio"),
    ("GLP", "Población"),
    
    ("Gasolina aviación", "Total"),
    ("Gasolina aviación", "Transporte"),
    
    ("Gasolina motor", "Total"),
    ("Gasolina motor", "Agricultura"),
    ("Gasolina motor", "Ind. manufactureras"),
    ("Gasolina motor", "Suministro elec/gas/agua"),
    ("Gasolina motor", "Construcción"),
    ("Gasolina motor", "Comercio"),
    ("Gasolina motor", "Transporte"),
    ("Gasolina motor", "Servicios empresariales"),
    ("Gasolina motor", "Administración pública"),
    ("Gasolina motor", "Población"),
    
    ("Nafta ind+Solventes", "Total"),
    ("Nafta ind+Solventes", "Minas y canteras"),
    
    ("Queroseno", "Total"),
    ("Queroseno", "Población"),
    
    ("Turbo combustible", "Total"),
    ("Turbo combustible", "Transporte"),
]

multi_cols = pd.MultiIndex.from_tuples(cols, names=["Producto", "Actividad"])

# Data rows per year
data = {
    2018: [
        54.1,4.3,1.2,12.3,9.3,5.5,3.3,8.5,0.9,1.5,
        65.3,5.6,59.7,
        1787.0,121.0,54.2,100.1,223.1,547.0,137.2,101.9,294.2,36.4,9.4,
        1.4,1.4,
        4769.7,506.3,53.1,493.4,3667.0,15.8,
        4.2,4.2,
        159.8,8.0,4.8,2.7,106.4,
        4.3,3.9,
        254.3,4.2,7.3,4.6,6.4,8.4,41.9,13.9,12.9,113.1,
        48.6,46.5,
        80.0,78.8,
        74.8,37.1
    ],
    2019: [
        49.7,3.5,1.0,13.8,6.1,5.0,6.4,5.9,0.8,1.2,
        42.6,1.0,41.1,
        1458.1,91.4,49.6,84.8,168.9,364.6,102.5,92.3,266.7,28.8,6.9,
        1.0,1.0,
        4833.7,494.5,61.0,376.8,3201.0,12.8,
        5.3,5.3,
        170.2,0.9,6.0,6.3,115.5,
        2.6,2.6,
        232.1,3.6,6.7,5.3,6.0,7.3,40.3,12.0,13.0,106.1,
        44.5,40.8,
        73.4,72.5,
        82.1,25.7
    ],
    2020: [
        40.2,3.1,0.8,9.6,6.9,4.2,3.3,6.1,0.3,0.2,
        36.4,0.8,27.2,
        1112.6,77.1,46.7,50.5,74.0,330.5,93.4,74.2,204.4,9.0,6.1,
        4.0,4.0,
        4793.3,530.2,57.2,416.6,3731.5,10.1,
        14.4,14.4,
        153.5,5.2,7.3,3.4,111.8,
        2.5,1.7,
        203.9,2.4,5.7,3.0,3.8,4.9,29.7,5.1,7.7,120.2,
        40.3,39.1,
        74.0,73.0,
        122.4,21.3
    ],
    2021: [
        37.3,2.0,1.1,5.5,3.0,3.9,4.7,7.2,1.2,0.1,
        28.0,0.8,27.2,
        1185.7,68.0,46.5,47.0,57.2,550.1,68.5,64.3,159.0,7.9,7.9,
        None,None,
        4686.4,527.5,31.9,120.3,3917.9,7.9,
        8.0,8.0,
        162.3,5.2,3.3,3.0,129.1,
        2.5,2.5,
        250.2,2.5,4.9,2.8,3.6,4.7,28.8,5.3,7.6,127.1,
        48.3,46.4,
        62.1,60.1,
        86.7,20.6
    ],
    2022: [
        39.1,3.0,0.9,9.5,2.8,4.9,1.4,10.8,0.3,0.1,
        10.6,0.4,10.1,
        1157.3,78.1,44.1,53.2,54.6,595.0,55.2,32.0,134.4,7.6,7.7,
        None,None,
        4240.6,476.2,32.2,128.3,3531.5,4.8,
        10.7,10.7,
        179.1,5.0,3.6,2.3,140.3,
        3.0,1.3,
        343.9,2.3,5.5,2.4,3.4,3.9,15.5,6.1,9.7,236.9,
        47.7,46.6,
        59.9,59.0,
        151.1,15.0
    ]
}

tabla_10_11 = pd.DataFrame.from_dict(data, orient="index", columns=multi_cols)
tabla_10_11.index.name = "Año"
tabla_10_11.T

Año                                                    2018    2019    2020  \
Producto                   Actividad                                          
Aceites/grasas lubricantes Total                       54.1    49.7    40.2   
                           Agricultura                  4.3     3.5     3.1   
                           Minas y canteras             1.2     1.0     0.8   
                           Industria azucarera         12.3    13.8     9.6   
                           Ind. manufactureras          9.3     6.1     6.9   
                           Construcción                 5.5     5.0     4.2   
                           Comercio                     3.3     6.4     3.3   
                           Transporte                   8.5     5.9     6.1   
                           Servicios empresariales      0.9     0.8     0.3   
                           Población                    1.5     1.2     0.2   
Asfalto                    Total                       65.3    42.6    36.4   
                           Ind. manufactureras          5.6     1.0     0.8   
                           Construcción                59.7    41.1    27.2   
Diésel                     Total                     1787.0  1458.1  1112.6   
                           Agricultura                121.0    91.4    77.1   
                           Minas y canteras            54.2    49.6    46.7   
                           Industria azucarera        100.1    84.8    50.5   
                           Ind. manufactureras        223.1   168.9    74.0   
                           Suministro elec/gas/agua   547.0   364.6   330.5   
                           Construcción               137.2   102.5    93.4   
                           Comercio                   101.9    92.3    74.2   
                           Transporte                 294.2   266.7   204.4   
                           Servicios empresariales     36.4    28.8     9.0   
                           Administración pública       9.4     6.9     6.1   
Coque combustible          Total                        1.4     1.0     4.0   
                           Ind. manufactureras          1.4     1.0     4.0   
Fuel oil                   Total                     4769.7  4833.7  4793.3   
                           Minas y canteras           506.3   494.5   530.2   
                           Industria azucarera         53.1    61.0    57.2   
                           Ind. manufactureras        493.4   376.8   416.6   
                           Suministro elec/gas/agua  3667.0  3201.0  3731.5   
                           Construcción                15.8    12.8    10.1   
Gas combustible            Total                        4.2     5.3    14.4   
                           Ind. manufactureras          4.2     5.3    14.4   
GLP                        Total                      159.8   170.2   153.5   
                           Minas y canteras             8.0     0.9     5.2   
                           Ind. manufactureras          4.8     6.0     7.3   
                           Comercio                     2.7     6.3     3.4   
                           Población                  106.4   115.5   111.8   
Gasolina aviación          Total                        4.3     2.6     2.5   
                           Transporte                   3.9     2.6     1.7   
Gasolina motor             Total                      254.3   232.1   203.9   
                           Agricultura                  4.2     3.6     2.4   
                           Ind. manufactureras          7.3     6.7     5.7   
                           Suministro elec/gas/agua     4.6     5.3     3.0   
                           Construcción                 6.4     6.0     3.8   
                           Comercio                     8.4     7.3     4.9   
                           Transporte                  41.9    40.3    29.7   
                           Servicios empresariales

In [48]:
# Tabla_10_12_Consumo_Hogares
tabla_10_12 = pd.DataFrame({
    "Leña (Mm³)": [17.2, 16.7, 16.2, 14.0, 15.1],
    "Electricidad (GWh)": [9256.9, 9428.0, 8515.7, 8056.5, 9038.3],
    "GLP (Mt)": [115.5, 111.8, 129.1, 140.3, None],
    "Queroseno (Mt)": [72.5, 73.0, 60.1, 59.0, None],
    "Alcohol desnaturalizado (Mhl)": [165.3, 167.0, 150.5, 139.4, 136.8],
    "Gas manufacturado (MMm³)": [110.5, 113.7, 109.6, 108.0, 110.2],
    "Carbón vegetal (Mt)": [6.9, 7.7, 6.4, 5.3, 6.7],
}, index=[2019, 2020, 2021, 2022, 2023])
tabla_10_12.index.name = "Año"
tabla_10_12

,Leña (Mm³),Electricidad (GWh),GLP (Mt),Queroseno (Mt),Alcohol desnaturalizado (Mhl),Gas manufacturado (MMm³),Carbón vegetal (Mt)
Año,,,,,,,
2019,17.2,9256.9,115.5,72.5,165.3,110.5,6.9
2020,16.7,9428.0,111.8,73.0,167.0,113.7,7.7
2021,16.2,8515.7,129.1,60.1,150.5,109.6,6.4
2022,14.0,8056.5,140.3,59.0,139.4,108.0,5.3
2023,15.1,9038.3,NaN,NaN,136.8,110.2,6.7


In [49]:
# Tabla_10_13_Indicadores_Electricidad
tabla_10_13 = pd.DataFrame({
    "Consumo prom. mensual total (kWh/cliente)": [304.8, 291.5, 266.8, 254.9, 281.1],
    "Consumo prom. mensual residencial": [191.1, 193.4, 175.4, 164.0, 182.2],
    "Generación bruta per cápita (kWh/hab)": [1849.6, 1830.9, 1741.2, 1652.3, None],
    "Nivel de electrificación (%)": [99.98, 99.98, 99.98, 99.98, 99.98],
}, index=[2019, 2020, 2021, 2022, 2023])
tabla_10_13.index.name = "Año"
tabla_10_13

,Consumo prom. mensual total (kWh/cliente),Consumo prom. mensual residencial,Generación bruta per cápita (kWh/hab),Nivel de electrificación (%)
Año,,,,
2019,304.8,191.1,1849.6,99.98
2020,291.5,193.4,1830.9,99.98
2021,266.8,175.4,1741.2,99.98
2022,254.9,164.0,1652.3,99.98
2023,281.1,182.2,NaN,99.98


In [50]:
# ---------------- Tabla 10.14 · Balance de electricidad (enero‑diciembre 2023) --------------
tabla_10_14 = pd.DataFrame({
    "2022 (GWh)": [
        18322.8, 15732.1, 15027.7, 588.7, 115.7,
        2590.7, 2590.7,
        18322.8, 14862.6, 5971.5, 1590.2, 8891.1, 8056.5, 834.6,
        3427.7, 568.1, 2859.5,
        32.5
    ],
    "2023 (GWh)": [
        19825.0, 15331.1, 14714.1, 458.0, 159.0,
        4493.8, 4493.8,
        19825.0, 16051.3, 6446.6, 1409.5, 9604.8, 9038.3, 566.5,
        3716.5, 635.9, 3080.6,
        57.1
    ],
    "Variación Absoluta (GWh)": [
        1502.2, -401.0, -313.6, -130.7, 43.3,
        1903.1, 1903.1,
        1502.2, 1188.7, 475.1, -180.7, 713.7, 981.8, -268.1,
        288.8, 67.8, 221.0,
        24.6
    ],
    "Variación %": [
        108.2, 97.5, 97.9, 77.8, 137.4,
        173.5, 173.5,
        108.2, 108.0, 108.0, 88.6, 108.0, 112.2, 67.9,
        108.4, 111.9, 107.7,
        175.6
    ]
}, index=[
    "Fuentes",
    "Empresas servicio público ‑ total",
    "  └─ Empresas de servicio público",
    "  └─ Empresas autoproductoras",
    "  └─ Otros",
    "Importación de energía eléctrica",
    "  (Generación móvil)",
    "Destinos",
    "Consumo",
    "  └─ Consumo estatal",
    "    └─ Insumo",
    "  └─ Consumo privado",
    "    └─ Residencial",
    "    └─ Privado",
    "Pérdidas",
    "  └─ Pérdidas en transmisión",
    "  └─ Pérdidas en distribución",
    "Desbalance"
])

tabla_10_14.index.name = "Descripción"
tabla_10_14


,2022 (GWh),2023 (GWh),Variación Absoluta (GWh),Variación %
Descripción,,,,
Fuentes,18322.8,19825.0,1502.2,108.2
Empresas servicio público ‑ total,15732.1,15331.1,-401.0,97.5
└─ Empresas de servicio público,15027.7,14714.1,-313.6,97.9
└─ Empresas autoproductoras,588.7,458.0,-130.7,77.8
└─ Otros,115.7,159.0,43.3,137.4
Importación de energía eléctrica,2590.7,4493.8,1903.1,173.5
(Generación móvil),2590.7,4493.8,1903.1,173.5
Destinos,18322.8,19825.0,1502.2,108.2
Consumo,14862.6,16051.3,1188.7,108.0


In [51]:
tabla_10_15 = pd.DataFrame({
    "Total (GWh)":               [20705.6, 19070.9, 17965.5, 15732.1, 15331.1],
    "Empresas servicio público": [19861.2, 18156.8, 17104.9, 15027.7, 14714.1],
    "Autoproductores - Total"  : [  841.9,   898.2,   752.6,   588.7,   458.0],
    "  └─ Industria azucarera":   [  519.0,   546.9,   432.4,   276.1,   203.2],
    "  └─ Industria del níquel":  [  322.9,   351.3,   320.2,   312.6,   254.8],
    "Otros (GWh)":               [    2.5,    15.9,   108.0,   115.7,   159.0],
}, index=[2019, 2020, 2021, 2022, 2023])

tabla_10_15.index.name = "Año"
tabla_10_15

,Total (GWh),Empresas servicio público,Autoproductores - Total,└─ Industria azucarera,└─ Industria del níquel,Otros (GWh)
Año,,,,,,
2019,20705.6,19861.2,841.9,519.0,322.9,2.5
2020,19070.9,18156.8,898.2,546.9,351.3,15.9
2021,17965.5,17104.9,752.6,432.4,320.2,108.0
2022,15732.1,15027.7,588.7,276.1,312.6,115.7
2023,15331.1,14714.1,458.0,203.2,254.8,159.0


In [52]:
# ----------------- Tabla 10.16 · Generación bruta por tipo de planta -----------------
tabla_10_16 = pd.DataFrame({
    "Total (GWh)":                    [20705.6, 19070.9, 17965.5, 15732.1, 15331.1],
    "Generación térmica - Termoeléctricas (GWh)": [12664.8, 11806.8, 11014.2, 9446.8, 9128.4],
    "Generación térmica - Autoproductores (GWh)": [  841.9,   898.2,   752.6,   588.7,   458.0],
    "Turbinas de gas (GWh)":          [2449.8, 2014.9, 1580.4, 1946.3, 2498.7],
    "Grupos electrógenos interconectados (GWh)": [4371.7, 3962.4, 4125.6, 3272.9, 2743.8],
    "Renovables - Hidroeléctricas (GWh)": [124.5, 111.9, 119.7, 120.2, 106.5],
    "Renovables - Fotovoltaica (GWh)":   [250.5, 260.8, 265.0, 241.5, 236.7],
    "Otros (biogás, emergentes) (GWh)": [  2.5,   15.9,  108.0,  115.7,  159.0],
}, index=[2019, 2020, 2021, 2022, 2023])
tabla_10_16.index.name = "Año"

# ------------- Tabla 10.17 · Consumo específico de combustible -----------------
tabla_10_17 = pd.DataFrame({
    "Total (g/kWh)":        [257.7, 258.2, 265.6, 271.1, 271.5],
    "Termoeléctricas":      [276.1, 272.0, 281.2, 289.8, 287.4],
    "Turbinas de gas":      [417.9, 407.2, 429.5, 400.5, 410.0],
    "Grupos electrógenos - Diesel":        [216.3, 216.3, 216.7, 217.5, 217.3],
    "Grupos electrógenos - Aisladas":      [235.8, 228.8, 232.2, 241.8, 242.7],
    "Tecnología nueva":     [218.0, 216.6, 218.2, 218.5, 222.6],
}, index=[2018, 2019, 2020, 2021, 2022])
tabla_10_17.index.name = "Año"

In [53]:
tabla_10_18 = pd.DataFrame({
    "Total (MW)": [6507.8, 6660.5, 6767.3, 6235.2, 5369.2],
    "Termoeléctricas (MW)": [2498.0, 2498.0, 2608.0, 2548.0, 1993.0],
    "Turbinas de gas (MW)": [580.0, 580.0, 580.0, 580.0, 580.0],
    "Plantas diésel (MW)": [114.1, 111.2, 111.2, 111.2, 112.2],
    "Plantas diésel aisladas (MW)": [106.1, 105.2, 104.3, 109.1, 171.8],
    "Tecnología nueva (MW)": [2527.5, 2515.0, 2488.4, 2045.5, 1667.1],
    "Hidroeléctricas (MW)": [64.0, 64.6, 64.6, 64.7, 64.7],
    "Parques eólicos + FV (MW)": [159.2, 221.5, 241.8, 263.0, 266.7],
    "Otras generación térmica (MW)": [458.9, 565.0, 569.0, 513.7, 513.7],
}, index=[2019, 2020, 2021, 2022, 2023])
tabla_10_18.index.name = "Año"

# ---------------- Tabla 10.19 · Consumo de energía eléctrica ----------------
tabla_10_19 = pd.DataFrame({
    "Total (GWh)": [21155.0, 20472.5, 19350.2, 18322.8, 19825.0],
    "Estatal (GWh)": [8469.6, 7617.7, 7211.4, 6838.7, 7070.2],
    "Industria (GWh)": [4873.9, 4512.0, 3852.6, 3349.3, 3851.4],
    "Insumo (GWh)": [1943.8, 1998.5, 1859.5, 1590.2, 1409.5],
    "Construcción (GWh)": [91.9, 75.6, 129.0, 78.7, 61.0],
    "Agropecuario (GWh)": [304.2, 230.0, 275.9, 177.8, 176.7],
    "Transporte (GWh)": [288.2, 256.6, 265.9, 258.9, 232.1],
    "Comercio (GWh)": [456.6, 466.3, 390.3, 366.5, 405.7],
    "Otros (GWh)": [2454.8, 2077.3, 2297.6, 2607.4, 2343.3],
    "Población (GWh)": [9256.9, 9428.0, 8515.7, 8056.5, 9038.3],
    "Pérdidas (GWh)": [3428.5, 3426.8, 3623.1, 3427.7, 3716.5],
}, index=[2019, 2020, 2021, 2022, 2023])
tabla_10_19.index.name = "Año"

In [58]:
#####################################################
# 0)  Cargar TODAS las tablas en un dict            #
#####################################################
DB = {
    "t10_03": tabla_10_3,
    "t10_04": tabla_10_4,
    "t10_05": tabla_10_5,
    "t10_06": tabla_10_6,
    "t10_07": tabla_10_7,
    "t10_10": tabla_10_10,
    "t10_11": tabla_10_11,
    "t10_12": tabla_10_12,
    "t10_14": tabla_10_14,
    "t10_16": tabla_10_16,
    "t10_18": tabla_10_18,
    "t10_19": tabla_10_19,
}

#####################################################
# 1)  Diccionario maestro de nodos                  #
#####################################################
labels = []           # nombres únicos
def node(name):
    if name not in labels:
        labels.append(name)
    return labels.index(name)

#####################################################
# 2)  Función auxiliar para agregar enlaces         #
#####################################################
source, target, value, color = [], [], [], []
def add_link(src, dst, val, col="#cccccc"):
    if pd.isna(val) or val == 0:           # ignora vacíos
        return
    source.append(node(src))
    target.append(node(dst))
    value.append(round(val, 2))
    color.append(col)

#####################################################
# 3)  AÑO que quieres graficar                      #
#####################################################
YR = 2022

#####################################################
# 4)  POV: recorrer las tablas y mapear             #
#####################################################
# ---- Producción primaria -------------------------
for rub in ["Petróleo (Mt)", "Gas natural (MMm³)", "Leña (Mm³)",
            "Productos de caña (Mt)", "Hidro‑energía (GWh)"]:
    v = DB["t10_03"].loc[YR, rub]
    add_link(rub, "Energía Primaria", v, "#8dd3c7")

# ---- Importaciones -------------------------------
imp_map = {
    "Petróleo crudo (kt)": "Petróleo (Mt)",
    "Derivados del petróleo (kt)": "Derivados petróleo",
    "Electricidad (GWh)": "Electricidad bruta",
}
for rub_db, rub_node in imp_map.items():
    v = DB["t10_06"].loc[YR, rub_db]
    add_link("Importaciones", rub_node, v, "#ffffb3")

# ---- Producción secundaria -----------------------
sec_table = DB["t10_04"].loc[YR]
for col in sec_table.index:
    add_link("Transformación", col, sec_table[col], "#bebada")

# ---- Derivados prod. -----------------------------
for col in DB["t10_05"].columns[1:]:   # skip Total
    v = DB["t10_05"].loc[YR, col]
    add_link("Refinería", col, v, "#fb8072")

# ---- Consumo electricidad sectorial --------------
cons_elec = DB["t10_19"].loc[YR]
for sector in ["Estatal (GWh)", "Industria (GWh)", "Transporte (GWh)",
               "Población (GWh)", "Pérdidas (GWh)"]:
    add_link("Red Eléctrica", sector.replace(" (GWh)", ""), cons_elec[sector], "#80b1d3")

# ---- Consumo hogar combustibles ------------------
home = DB["t10_12"].loc[YR]
for col in home.index:
    add_link(col.split()[0], "Hogares", home[col], "#fdb462")

# ---- Combustibles por sector (10.11) -------------
multi = DB["t10_11"]
if YR in multi.index:
    fila = multi.loc[YR]
    for prod, act in fila.index:  # MultiIndex
        if "Total" in act:
            add_link(prod, f"{prod} Usos", fila[(prod, act)], "#b3de69")

#####################################################
# 5)  Dibujar Sankey                                #
#####################################################
import plotly.graph_objects as go
fig = go.Figure(data=[go.Sankey(
    node=dict(label=labels, pad=15, thickness=20),
    link=dict(source=source, target=target, value=value, color=color)
)])
fig.update_layout(title=f"Balance energético Cuba {YR}", font_size=10, width=1000, height=800)
fig.show()


In [70]:
# Suponiendo que ya tienes los DataFrames:
crudo_prod = tabla_10_3.loc[2022,"Petróleo (Mt)"]
crudo_imp  = tabla_10_6.loc[2022,"Petróleo crudo (kt)"]
print("Crudo total disponible:", crudo_prod + crudo_imp, "kt")

fuel_stock = tabla_10_5.loc[2022,"Fuel oil (kt)"] + tabla_10_7.loc[2022,"Fuel oil (kt)"]
print("Fuel-oil stock:", fuel_stock, "kt")

fuel_to_power = tabla_10_11.loc[2022,("Fuel oil","Suministro elec/gas/agua")]
print("Fuel-oil a eléctricas:", fuel_to_power, "kt")

gwh_termo = tabla_10_16.loc[2022,"Generación térmica - Termoeléctricas (GWh)"]
print("Salida eléctrica termo:", gwh_termo, "GWh")


Crudo total disponible: 4768.6 kt
Fuel-oil stock: 1607.9 kt
Fuel-oil a eléctricas: 3531.5 kt
Salida eléctrica termo: 9446.8 GWh


$$ MMm³\cdot 10⁶ \cdot \frac{38MJ}{m³}$$

$$ \frac {2373.9 kt \cdot \frac {10⁶kg}{kt} \cdot \frac{42 MJ}{kg}} {\frac {7000 kcal}{1tcc} \cdot \frac {4.18 kJ}{1 kcal}\cdot \frac{1MJ}{1000kJ}} $$

In [ ]:
# Conversión de MMm³ a kton para gas natural densidad 0.72 kg/m³
gas_mass = tabla_10_3.loc[2022,"Gas natural (MMm³)"]*1e6*0.72/1000/1000

tabla_10_3.loc[2022,"Gas natural (MMm³)"]*1e6*38


np.float64(32053000000.0)

In [102]:
# Conversion de crudo
tabla_10_3.loc[2022,"Petróleo (Mt)"] * 1e3 * 41.84 / (10000 * 4.184/1000)/1000

np.float64(2373.9)

In [95]:
(7000 * 4.184/1000)

29.288

Masa gas 2022: 607320.0 kt
Energía gas: 20891.81 PJ
Gas en kt tcc: 713323.1


In [108]:
#!/usr/bin/env python
# cuba_sankey_final.py  –  demand-pull Sankey for Cuba 2022
# -----------------------------------------------------------
import plotly.graph_objects as go
import pandas as pd

# -----------------------------------------------------------
# 1 · CONSTANTS  (MJ per unit, calibrated to match ONEI)
# -----------------------------------------------------------
PCI = dict(
    # Primaries
    crude_MJt   = 37.0e3,          # 37 MJ/kg  (crudo cubano)
    gas_MJkg    = 34.4e3,          # 34.4 MJ/kg
    bagazo_MJkg =  2.1e3,
    lena_MJkg   =  2.5e3,
    # Secondaries
    glp_MJt     = 46.1e3,
    gasman_MJm3 = 13.4,            # MJ per m3
    gasolina_MJt= 44.5e3,
    diesel_MJt  = 43.0e3,
    turbo_MJt   = 43.5e3,
    quer_MJt    = 43.0e3,
    fuel_MJt    = 41.0e3,
)

MJ_PER_TCC = 29.288e3   # ONEI base 7 000 kcal kg-1

# -----------------------------------------------------------
# 2 · TABLE VALUES 2022  (paste directly from 10.11 / 10.19)
# -----------------------------------------------------------
VAL = dict(
    # --- Final uses (physical units) -----------------------
    home_glp_kt   = 140.3,
    home_gasman_MMm3 = 110.0,
    home_lena_Mm3 = 14.0,
    home_quer_kt  = 59.0,
    home_elec_GWh = 8056,

    transp_gaso_kt= 343.9,
    transp_diesel_kt = 134.4,
    transp_turbo_kt  = 151.1,
    transp_elec_GWh  = 232,

    ind_fuel_kt   = 128.3,
    ind_diesel_kt = 54.6,
    ind_bagazo_kt = 312.6,     # bagazo quemado en ingenios
    ind_elec_GWh  = 3851,

    service_fuel_kt = 476.2,
    service_elec_GWh= 6446,

    losses_grid_GWh = 3716,    # pérdidas transm+dist
    # --- Secondary stocks / production ---------------------
    deriv_prod_kt = 1618.9,
    glp_prod_kt   = 27.5,
    glp_imp_kt    = 187.5,
    gasman_prod_MMm3  = 181.2,
    fuel_imp_kt   = 679.2,
    diesel_imp_kt = 876.0,
    gasolina_imp_kt = 380.7,
    # Electricity generation & import (10.16 / 10.14)
    elec_thermo_GWh = 9446.8,
    elec_bagazo_GWh = 276.1,
    elec_hydro_GWh  = 106.5,
    elec_solar_GWh  = 236.7,
    elec_wind_GWh   = 159.0,
    elec_import_GWh = 4493.8,
    # --- Primary production & imports ----------------------
    crude_nat_kt = 2373.9,
    crude_imp_kt = 2394.7,
    gas_nat_MMm3 = 843.5,
    bagazo_nat_kt= 2551.2,
    lena_nat_Mm3 = 659.1,
)

# handy lambdas
PJ = {}
PJ['home'] = (VAL['home_glp_kt'] * PCI['glp_MJt'] +
              VAL['home_gasman_MMm3']*1e6*PCI['gasman_MJm3'] +
              VAL['home_lena_Mm3']*1e6*0.65*PCI['lena_MJkg'] +
              VAL['home_quer_kt']*PCI['quer_MJt'] +
              VAL['home_elec_GWh']*3.6e3) / 1e6

PJ['transport'] = (VAL['transp_gaso_kt']*PCI['gasolina_MJt'] +
                   VAL['transp_diesel_kt']*PCI['diesel_MJt'] +
                   VAL['transp_turbo_kt']*PCI['turbo_MJt'] +
                   VAL['transp_elec_GWh']*3.6e3) / 1e6

PJ['industry'] = (VAL['ind_fuel_kt']*PCI['fuel_MJt'] +
                  VAL['ind_diesel_kt']*PCI['diesel_MJt'] +
                  VAL['ind_bagazo_kt']*PCI['bagazo_MJkg'] +
                  VAL['ind_elec_GWh']*3.6e3) / 1e6

PJ['services'] = (VAL['service_fuel_kt']*PCI['fuel_MJt'] +
                  VAL['service_elec_GWh']*3.6e3) / 1e6

PJ['losses'] = VAL['losses_grid_GWh']*3.6/1e3

# -----------------------------------------------------------
# 3 · Build Sankey ledger  (right → left)
# -----------------------------------------------------------
links = []          # list of dicts (source, target, value PJ)
nodes = []          # unique list for Plotly
def idx(name):
    if name not in nodes: nodes.append(name)
    return nodes.index(name)
def add(src,tgt,val):
    if val>0:
        links.append(dict(source=idx(src),target=idx(tgt),value=round(val,2)))

# --- layer: final uses -------------------------------------
add('GLP residencial','Cocción hogar',VAL['home_glp_kt']*PCI['glp_MJt']/1e6)
add('Gas de ciudad','Cocción hogar',VAL['home_gasman_MMm3']*1e6*PCI['gasman_MJm3']/1e6)
add('Leña','Cocción hogar',VAL['home_lena_Mm3']*1e6*0.65*PCI['lena_MJkg']/1e6)
add('Queroseno hogar','Cocción hogar',VAL['home_quer_kt']*PCI['quer_MJt']/1e6)
add('Red eléctrica','Cocción hogar',VAL['home_elec_GWh']*3.6e3/1e6)

add('Gasolina','Transporte',VAL['transp_gaso_kt']*PCI['gasolina_MJt']/1e6)
add('Diésel transp','Transporte',VAL['transp_diesel_kt']*PCI['diesel_MJt']/1e6)
add('Turbocomb','Transporte',VAL['transp_turbo_kt']*PCI['turbo_MJt']/1e6)
add('Red eléctrica','Transporte',VAL['transp_elec_GWh']*3.6e3/1e6)

add('Fuel industria','Industria',VAL['ind_fuel_kt']*PCI['fuel_MJt']/1e6)
add('Diésel ind','Industria',VAL['ind_diesel_kt']*PCI['diesel_MJt']/1e6)
add('Bagazo propio','Industria',VAL['ind_bagazo_kt']*PCI['bagazo_MJkg']/1e6)
add('Red eléctrica','Industria',VAL['ind_elec_GWh']*3.6e3/1e6)

add('Fuel estatal','Servicios',VAL['service_fuel_kt']*PCI['fuel_MJt']/1e6)
add('Red eléctrica','Servicios',VAL['service_elec_GWh']*3.6e3/1e6)

add('Red eléctrica','Pérdidas red',VAL['losses_grid_GWh']*3.6e3/1e6)

# --- layer: secondary stocks -> final fuels ---------------
sec_GLPe   = (VAL['home_glp_kt'] + VAL['glp_imp_kt'])*PCI['glp_MJt']/1e6
add('GLP stock','GLP residencial',sec_GLPe)

sec_gasman = VAL['gasman_prod_MMm3']*1e6*PCI['gasman_MJm3']/1e6
add('Planta gas','Gas de ciudad',sec_gasman)

sec_fuel   = (VAL['fuel_imp_kt']+VAL['ind_fuel_kt']+VAL['service_fuel_kt']+
              VAL['elec_thermo_GWh']*3.6e3/PCI['fuel_MJt'])*PCI['fuel_MJt']/1e6
add('Fuel stock','Fuel industria',VAL['ind_fuel_kt']*PCI['fuel_MJt']/1e6)
add('Fuel stock','Fuel estatal',VAL['service_fuel_kt']*PCI['fuel_MJt']/1e6)
add('Fuel stock','Plantas térmicas',VAL['elec_thermo_GWh']*3.6e3/1e6)

add('Diésel stock','Diésel ind',VAL['ind_diesel_kt']*PCI['diesel_MJt']/1e6)
add('Diésel stock','Diésel transp',VAL['transp_diesel_kt']*PCI['diesel_MJt']/1e6)
add('Diésel stock','Plantas térmicas',0)  # simplificado

# Electricity generation mix
add('Plantas térmicas','Red eléctrica',VAL['elec_thermo_GWh']*3.6e3/1e6)
add('Bagazo autoproduct','Red eléctrica',VAL['elec_bagazo_GWh']*3.6e3/1e6)
add('Hidráulica','Red eléctrica',VAL['elec_hydro_GWh']*3.6e3/1e6)
add('Solar FV','Red eléctrica',VAL['elec_solar_GWh']*3.6e3/1e6)
add('Eólica','Red eléctrica',VAL['elec_wind_GWh']*3.6e3/1e6)
add('Electricidad importada','Red eléctrica',VAL['elec_import_GWh']*3.6e3/1e6)

# --- layer: primary supply -------------------------------
add('Crudo disponible','Fuel stock',
    (VAL['crude_nat_kt']+VAL['crude_imp_kt'])*PCI['crude_MJt']/1e6*0.30)  # ≈30% destilado a fuel
add('Crudo disponible','Diésel stock',
    (VAL['crude_nat_kt']+VAL['crude_imp_kt'])*PCI['crude_MJt']/1e6*0.25)  # simplificado
add('Crudo disponible','GLP stock',VAL['glp_prod_kt']*PCI['glp_MJt']/1e6)

add('Planta de azúcar','Bagazo propio',VAL['bagazo_nat_kt']*PCI['bagazo_MJkg']/1e6)
add('Gas natural','Plantas térmicas',0)  # sin turbinas en 2022

add('Naturaleza','Crudo disponible',VAL['crude_nat_kt']*PCI['crude_MJt']/1e6)
add('Importaciones','Crudo disponible',VAL['crude_imp_kt']*PCI['crude_MJt']/1e6)
add('Naturaleza','Gas natural',VAL['gas_nat_MMm3']*1e6*PCI['gasman_MJm3']/1e6)  # usando mismo PCI
add('Naturaleza','Bagazo disponible',VAL['bagazo_nat_kt']*PCI['bagazo_MJkg']/1e6)
add('Naturaleza','Leña',VAL['lena_nat_Mm3']*1e6*0.65*PCI['lena_MJkg']/1e6)

# -----------------------------------------------------------
# 4 · Plotly Sankey
# -----------------------------------------------------------
fig = go.Figure(go.Sankey(
    arrangement="snap",
    valueformat=".2f",
    node=dict(label=nodes, pad=15, thickness=18,
              color=["#8f8f8f" if "Import" in n or n == "Naturaleza"
                     else "#e377c2" if "Eléct" in n
                     else "#1f77b4" if "Crudo" in n or "Fuel" in n or "Diésel" in n or "GLP" in n
                     else "#2ca02c" if "Bagazo" in n or "Leña" in n
                     else "#ff7f0e" if "Gas" in n and "Ciudad" in n
                     else "#d62728" if "Gasolina" in n or "Turbo" in n
                     else "#7f7f7f"]),
    ),
    link=dict(
        source=[l['source'] for l in links],
        target=[l['target'] for l in links],
        value =[l['value']  for l in links]
    )
)
fig.update_layout(title_text="Cuba – Balance Energético 2022 (PJ, demand-pull)",
                  font_size=11, height=900)
fig.show()


NameError: name 'n' is not defined